<a href="https://colab.research.google.com/github/sanja7s/AIIA/blob/main/MitigationGen_AI_Design.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup
#### Load the API key and relevant Python libaries.

In [ ]:
%%capture
!pip install langchain
!pip install faiss-cpu
!pip install openai==0.28
!pip install unstructured
!pip install python-dotenv
!pip install tiktoken

In [ ]:
from google.colab import files
import io
from dotenv import dotenv_values, load_dotenv, find_dotenv
import openai
import os
from copy import deepcopy
import json
import time
import ast

# env file
uploaded = files.upload()

Saving env_HumanRights to env_HumanRights


In [ ]:
# Get the first key from the uploaded dictionary
env_file_key = list(uploaded.keys())[0]

# Read the uploaded file
env_content = uploaded[env_file_key].decode('utf-8')

# Load the content into a variable
env_variables = dotenv_values(stream=io.StringIO(env_content))

api_key = env_variables['OPENAI_API_KEY']
openai.api_key = api_key

# Models

In [ ]:
def get_completion_from_messages(messages,
                                 model="gpt-4",
                                 temperature=0,
                                 max_tokens=600): #1100
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
        max_tokens=max_tokens, # the maximum number of tokens the model can ouptut
    )
    return response.choices[0].message["content"]

In [ ]:
def get_completion_and_token_count(messages,
                                 model="gpt-4",
                                 temperature=0,
                                 max_tokens=600):

    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )

    content = response.choices[0].message["content"]

    token_dict = {
    'prompt_tokens':response['usage']['prompt_tokens'],
    'completion_tokens':response['usage']['completion_tokens'],
    'total_tokens':response['usage']['total_tokens'],
    }

    return content, token_dict

# Iterate and Save Use Riskiness Results

# Functions

In [ ]:
def replace_key(d, old_key, new_key):
  """
  Replace `old_key` with `new_key` in dictionary `d`.
  The associated value is retained.
  """
  if old_key in d:
      d[new_key] = d.pop(old_key)
  return d

## Read In Prompt Result

In [ ]:
def read_prompt_output():
  print("Select the right input you need.")
  selected_prompt_uploaded = files.upload() # change this for other prompts

  # Get the first key from the uploaded dictionary
  file_key = list(selected_prompt_uploaded.keys())[0]

  # Read the uploaded file
  file_content = selected_prompt_uploaded[file_key].decode('utf-8')

  file_content_dict = ast.literal_eval(file_content)

  # file_content_dict_rel =

  return file_content_dict

# Read Risk Reports

In [ ]:
EUAIACT_risks = read_prompt_output()
# SDG_risks = read_prompt_output()
# HR_risks = read_prompt_output()

Select the right input you need.


Saving data_subset23.json to data_subset23.json


In [ ]:
EUAIACT_risks

[{'id': '15',
  'Details': ['Well-being',
   'Personal growth coaching',
   'Analyzing facial responses to personal growth exercises',
   'Personal growth apps, coaches',
   'Coaching clients'],
  'Description': 'The AI system intended to be used for personal growth coaching, with the capability of analyzing facial responses to personal growth exercises. The users of this AI system are personal growth apps and coaches, and the subjects are coaching clients.',
  'Classification_GPT': 'High Risk',
  'Reasoning': 'The AI system in question is used for personal growth coaching and analyzes facial responses, which can be considered a form of biometric data. According to the EU AI Act and its amendments, AI systems that use biometric data for identification or to make inferences about personal characteristics are classified as high risk. Therefore, this AI system falls under the high-risk category.',
  'AIActText': 'High-risk AI systems pursuant to Article 6(2) are the AI systems listed in a

# 1 EU AI Act MITIGATION PROMPT

In [ ]:
# Assuming you have the variables domain, purpose, aiCapability, aiUser, and aiSubject defined with appropriate values

# V1:   AI Users:      In any case, the mitigation strategy that you propose should be for the oridinary individuals who are potential users of the AI System (and not for e.g., developers). What can they do in their power if anything to mitigate some of the risks of the AI System and have it useful?
# V2:   Ai Subjects:          In any case, the mitigation strategy that you propose should be for the oridinary individuals who are potential subjects of the AI System (and not for e.g., developers). What can they do in their power if anything to mitigate some of the risks of the AI System and have it still useful?


MESSAGES = [
    {
        'role': 'system',
        'content': """You are an experienced regulatory compliance specialists who works in the field of AI technology regulation. You are thoughtful, decisive, experienced and conscientious.
        You have access to the entirety of the EU AI Act and its amendments, which outline how various AI technologies are to be regulated and risk-classified within the European Union."""
    },
    {
        'role': 'user',
        'content': """

        If the AI System provided in input is classified as high risk according to the EU AI Act (classification is given in the input field "Classification"), consider carefully the system's output, as well as the reasoning, and propose a mitigation strategy that would make the system not anymore classified as high risk.
        If the AI System is not classified as high risk (i.e., if the input field "Classification" says "Not Classified as High Risk or Unacceptable Risk"), you can still recommend a mitigation strategy that would make the system more responsible.
        If the fields "Classification" and "Classification_GPT" disagree, consider that field "Classification" as correct; it means that annotators have manually corrected the GPT classification.

        In any case, the mitigation strategy that you propose should be for the oridinary individuals who are potential subjects of the AI System (and not for e.g., developers). What can they do in their power if anything to mitigate some of the risks of the AI System and have it still useful?

        Moreover, label the type of the mitigation strategy proposed considering the definitions of 4 possible labels below:
          Avoidance: take measures to avoid the risk from occurring.
          Reduction: reduce the likelihood of a risk happening or the impact should it occur.
          Transference: pass the risk consequence to a third party.
          Acceptance: accepting the risk as it stands (this means, no mitigation strategy was found).

        It is of utmost importance to exercise precision and make accurate judgments when classifying the risk associated with the AI system.
        Please carefully consider all the regulations listed below during the risk classification of the AI system:

        The relevant portions of the Act for what is unacceptable:
        5.2.2. PROHIBITED ARTIFICIAL INTELLIGENCE PRACTICES (TITLE II)
        Title II establishes a list of prohibited AI. The regulation follows a risk-based approach,
        differentiating between uses of AI that create (i) an unacceptable risk, (ii) a high risk, and (iii)
        low or minimal risk. The list of prohibited practices in Title II comprises all those AI systems
        whose use is considered unacceptable as contravening Union values, for instance by violating
        fundamental rights. The prohibitions covers practices that have a significant potential to
        manipulate persons through subliminal techniques beyond their consciousness or exploit vulnerabilities
        of specific vulnerable groups such as children or persons with disabilities in
        order to materially distort their behaviour in a manner that is likely to cause them or another
        person psychological or physical harm. Other manipulative or exploitative practices affecting
        adults that might be facilitated by AI systems could be covered by the existing data
        protection, consumer protection and digital service legislation that guarantee that natural
        persons are properly informed and have free choice not to be subject to profiling or other
        practices that might affect their behaviour. The proposal also prohibits AI-based social
        scoring for general purposes done by public authorities. Finally, the use of ‘real time’ remote
        biometric identification systems in publicly accessible spaces for the purpose of law
        enforcement is also prohibited unless certain limited exceptions apply.

        PROHIBITED ARTIFICIAL INTELLIGENCE PRACTICES
        Article 5
        1. The following artificial intelligence practices shall be prohibited:
        (a) the placing on the market, putting into service or use of an AI system that
        deploys subliminal techniques beyond a person’s consciousness in order to
        materially distort a person’s behaviour in a manner that causes or is likely to
        cause that person or another person physical or psychological harm;
        (b) the placing on the market, putting into service or use of an AI system that
        exploits any of the vulnerabilities of a specific group of persons due to their
        age, physical or mental disability, in order to materially distort the behaviour of
        a person pertaining to that group in a manner that causes or is likely to cause
        that person or another person physical or psychological harm;
        (c) the placing on the market, putting into service or use of AI systems by public
        authorities or on their behalf for the evaluation or classification of the
        trustworthiness of natural persons over a certain period of time based on their
        social behaviour or known or predicted personal or personality characteristics,
        with the social score leading to either or both of the following:
        (i) detrimental or unfavourable treatment of certain natural persons or whole
        groups thereof in social contexts which are unrelated to the contexts in
        which the data was originally generated or collected;
        (ii) detrimental or unfavourable treatment of certain natural persons or whole
        groups thereof that is unjustified or disproportionate to their social
        behaviour or its gravity;
        (d) the use of ‘real-time’ remote biometric identification systems in publicly
        accessible spaces for the purpose of law enforcement, unless and in as far as
        such use is strictly necessary for one of the following objectives:
        the targeted search for specific potential victims of crime, including
        missing children;
        (ii) the prevention of a specific, substantial and imminent threat to the life or
        physical safety of natural persons or of a terrorist attack;
        (iii) the detection, localisation, identification or prosecution of a perpetrator
        or suspect of a criminal offence referred to in Article 2(2) of Council
        Framework Decision 2002/584/JHA62 and punishable in the Member
        State concerned by a custodial sentence or a detention order for a
        maximum period of at least three years, as determined by the law of that
        Member State.
        2. The use of ‘real-time’ remote biometric identification systems in publicly accessible
        spaces for the purpose of law enforcement for any of the objectives referred to in
        paragraph 1 point d) shall take into account the following elements:
        (a) the nature of the situation giving rise to the possible use, in particular the
        seriousness, probability and scale of the harm caused in the absence of the use
        of the system;
        (b) the consequences of the use of the system for the rights and freedoms of all
        persons concerned, in particular the seriousness, probability and scale of those
        consequences.
        In addition, the use of ‘real-time’ remote biometric identification systems in publicly
        accessible spaces for the purpose of law enforcement for any of the objectives
        referred to in paragraph 1 point d) shall comply with necessary and proportionate
        safeguards and conditions in relation to the use, in particular as regards the temporal,
        geographic and personal limitations.
        3. As regards paragraphs 1, point (d) and 2, each individual use for the purpose of law
        enforcement of a ‘real-time’ remote biometric identification system in publicly
        accessible spaces shall be subject to a prior authorisation granted by a judicial
        authority or by an independent administrative authority of the Member State in
        which the use is to take place, issued upon a reasoned request and in accordance with
        the detailed rules of national law referred to in paragraph 4. However, in a duly
        justified situation of urgency, the use of the system may be commenced without an
        authorisation and the authorisation may be requested only during or after the use.
        The competent judicial or administrative authority shall only grant the authorisation
        where it is satisfied, based on objective evidence or clear indications presented to it,
        that the use of the ‘real-time’ remote biometric identification system at issue is
        necessary for and proportionate to achieving one of the objectives specified in
        paragraph 1, point (d), as identified in the request. In deciding on the request, the
        competent judicial or administrative authority shall take into account the elements
        referred to in paragraph 2.
        4. A Member State may decide to provide for the possibility to fully or partially
        authorise the use of ‘real-time’ remote biometric identification systems in publicly
        accessible spaces for the purpose of law enforcement within the limits and under the
        conditions listed in paragraphs 1, point (d), 2 and 3. That Member State shall lay
        down in its national law the necessary detailed rules for the request, issuance and
        exercise of, as well as supervision relating to, the authorisations referred to in
        paragraph 3. Those rules shall also specify in respect of which of the objectives listed
        in paragraph 1, point (d), including which of the criminal offences referred to in
        point (iii) thereof, the competent authorities may be authorised to use those systems
        for the purpose of law enforcement.

        The relevant portions of the Act for what is High risk:
        CLASSIFICATION OF AI SYSTEMS AS HIGH-RISK
        Article 6
        Classification rules for high-risk AI systems
        1. Irrespective of whether an AI system is placed on the market or put into service
        independently from the products referred to in points (a) and (b), that AI system shall
        be considered high-risk where both of the following conditions are fulfilled:
        (a) the AI system is intended to be used as a safety component of a product, or is
        itself a product, covered by the Union harmonisation legislation listed in Annex
        II;
        (b) the product whose safety component is the AI system, or the AI system itself as
        a product, is required to undergo a third-party conformity assessment with a
        view to the placing on the market or putting into service of that product
        pursuant to the Union harmonisation legislation listed in Annex II.
        2. In addition to the high-risk AI systems referred to in paragraph 1, AI systems
        referred to in Annex III shall also be considered high-risk.
        When assessing for the purposes of paragraph 1 whether an AI system poses a risk of
        harm to the health and safety or a risk of adverse impact on fundamental rights that is
        equivalent to or greater than the risk of harm posed by the high-risk AI systems already
        referred to in Annex III, the Commission shall take into account the
        following criteria:
        (a) the intended purpose of the AI system;
        (b) the extent to which an AI system has been used or is likely to be used;
        (c) the extent to which the use of an AI system has already caused harm to the
        health and safety or adverse impact on the fundamental rights or has given rise
        to significant concerns in relation to the materialisation of such harm or
        adverse impact, as demonstrated by reports or documented allegations
        submitted to national competent authorities;
        (d) the potential extent of such harm or such adverse impact, in particular in terms
        of its intensity and its ability to affect a plurality of persons;
        (e) the extent to which potentially harmed or adversely impacted persons are
        dependent on the outcome produced with an AI system, in particular because
        for practical or legal reasons it is not reasonably possible to opt-out from that
        outcome;
        (f) the extent to which potentially harmed or adversely impacted persons are in a
        vulnerable position in relation to the user of an AI system, in particular due to
        an imbalance of power, knowledge, economic or social circumstances, or age;
        (g) the extent to which the outcome produced with an AI system is easily
        reversible, whereby outcomes having an impact on the health or safety of
        persons shall not be considered as easily reversible;
        (h) the extent to which existing Union legislation provides for:
        (i) effective measures of redress in relation to the risks posed by an AI
        system, with the exclusion of claims for damages;
        (ii) effective measures to prevent or substantially minimise those risks.


        HIGH-RISK AI SYSTEMS REFERRED TO IN ARTICLE 6(2)
        High-risk AI systems pursuant to Article 6(2) are the AI systems listed in any of the following
        areas:
        1. Biometric identification and categorisation of natural persons:
        (a) AI systems intended to be used for the ‘real-time’ and ‘post’ remote biometric
        identification of natural persons;
        2. Management and operation of critical infrastructure:
        (a) AI systems intended to be used as safety components in the management and
        operation of road traffic and the supply of water, gas, heating and electricity.
        3. Education and vocational training:
        (a) AI systems intended to be used for the purpose of determining access or
        assigning natural persons to educational and vocational training institutions;
        (b) AI systems intended to be used for the purpose of assessing students in
        educational and vocational training institutions and for assessing participants in
        tests commonly required for admission to educational institutions.
        4. Employment, workers management and access to self-employment:
        (a) AI systems intended to be used for recruitment or selection of natural persons,
        notably for advertising vacancies, screening or filtering applications, evaluating
        candidates in the course of interviews or tests;
        (b) AI intended to be used for making decisions on promotion and termination of
        work-related contractual relationships, for task allocation and for monitoring
        and evaluating performance and behavior of persons in such relationships.
        5. Access to and enjoyment of essential private services and public services and
        benefits:
        (a) AI systems intended to be used by public authorities or on behalf of public
        authorities to evaluate the eligibility of natural persons for public assistance
        benefits and services, as well as to grant, reduce, revoke, or reclaim such
        benefits and services;
        (b) AI systems intended to be used to evaluate the creditworthiness of natural
        persons or establish their credit score, with the exception of AI systems put into
        service by small scale providers for their own use;
        (c) AI systems intended to be used to dispatch, or to establish priority in the
        dispatching of emergency first response services, including by firefighters and
        medical aid.
        6. Law enforcement:
        (a) AI systems intended to be used by law enforcement authorities for making
        individual risk assessments of natural persons in order to assess the risk of a
        natural person for offending or reoffending or the risk for potential victims of
        criminal offences;
        (b) AI systems intended to be used by law enforcement authorities as polygraphs
        and similar tools or to detect the emotional state of a natural person;
        EN 5 EN
        (c) AI systems intended to be used by law enforcement authorities to detect deep
        fakes as referred to in article 52(3);
        (d) AI systems intended to be used by law enforcement authorities for evaluation
        of the reliability of evidence in the course of investigation or prosecution of
        criminal offences;
        (e) AI systems intended to be used by law enforcement authorities for predicting
        the occurrence or reoccurrence of an actual or potential criminal offence
        based on profiling of natural persons as referred to in Article 3(4) of Directive
        (EU) 2016/680 or assessing personality traits and characteristics or past
        criminal behaviour of natural persons or groups;
        (f) AI systems intended to be used by law enforcement authorities for profiling of
        natural persons as referred to in Article 3(4) of Directive (EU) 2016/680 in the
        course of detection, investigation or prosecution of criminal offences;
        (g) AI systems intended to be used for crime analytics regarding natural persons,
        allowing law enforcement authorities to search complex related and unrelated
        large data sets available in different data sources or in different data formats in
        order to identify unknown patterns or discover hidden relationships in the data.
        7. Migration, asylum and border control management:
        (a) AI systems intended to be used by competent public authorities as polygraphs
        and similar tools or to detect the emotional state of a natural person;
        (b) AI systems intended to be used by competent public authorities to assess a risk,
        including a security risk, a risk of irregular immigration, or a health risk, posed
        by a natural person who intends to enter or has entered into the territory of a
        Member State;
        (c) AI systems intended to be used by competent public authorities for the
        verification of the authenticity of travel documents and supporting
        documentation of natural persons and detect non-authentic documents by
        checking their security features;
        (d) AI systems intended to assist competent public authorities for the examination
        of applications for asylum, visa and residence permits and associated
        complaints with regard to the eligibility of the natural persons applying for a
        status.
        8. Administration of justice and democratic processes:
        (a) AI systems intended to assist a judicial authority in researching and
        interpreting facts and the law and in applying the law to a concrete set of facts.

        Here are some important amendments to the EU AI Act: It is very important to consider them for the risk classification. Please read them carefully:
        Amendment 709
        Proposal for a regulation
        Annex III – paragraph 1 – introductory part
        High-risk AI systems pursuant to Article 6(2) are the AI systems listed in any of the following areas:
          The AI systems specifically refered to in under points 1 to 8a stand for critical use cases and are each considered to be high-risk AI systems pursuant to Article 6(2), provided that they fulfil the criteria set out in that Article:
        Amendment 710
        Proposal for a regulation
        Annex III – paragraph 1 – point 1 – introductory part
        1.  Biometric identification and categorisation of natural persons:
          1.  Biometric and biometrics-based systems
        Amendment 711
        Proposal for a regulation
        Annex III – paragraph 1 – point 1 – point a
        (a)  AI systems intended to be used for the ‘real-time’ and ‘post’ remote biometric identification of natural persons;
          (a)  AI systems intended to be used for biometric identification of natural persons, with the exception of those mentioned in Article 5;
        Amendment 712
        Proposal for a regulation
        Annex III – paragraph 1 – point 1 – point a a (new)
          (a a)  AI systems intended to be used to make inferences about personal characteristics of natural persons on the basis of biometric or biometrics-based data, including emotion recognition systems, with the exception of those mentioned in Article 5;
          Point 1 shall not include AI systems intended to be used for biometric verification whose sole purpose is to confirm that a specific natural person is the person he or she claims to be.
        Amendment 713
        Proposal for a regulation
        Annex III – paragraph 1 – point 2 – point a
        (a)  AI systems intended to be used as safety components in the management and operation of road traffic and the supply of water, gas, heating and electricity.
          (a)  AI systems intended to be used as safety components in the management and operation of road, rail and air traffic unless they are regulated in harmonisation or sectoral law.
        Amendment 714
        Proposal for a regulation
        Annex III – paragraph 1 – point 2 – point a a (new)
          (a a)  AI systems intended to be used as safety components in the management and operation of the supply of water, gas, heating, electricity and critical digital infrastructure;
        Amendment 715
        Proposal for a regulation
        Annex III – paragraph 1 – point 3 – point a
        (a)  AI systems intended to be used for the purpose of determining access or assigning natural persons to educational and vocational training institutions;
          (a)  AI systems intended to be used for the purpose of determining access or materially influence decisions on admission or assigning natural persons to educational and vocational training institutions;
        Amendment 716
        Proposal for a regulation
        Annex III – paragraph 1 – point 3 – point b
        (b)  AI systems intended to be used for the purpose of assessing students in educational and vocational training institutions and for assessing participants in tests commonly required for admission to educational institutions.
          (b)  AI systems intended to be used for the purpose of assessing students in educational and vocational training institutions and for assessing participants in tests commonly required for admission to those institutions;
        Amendment 717
        Proposal for a regulation
        Annex III – paragraph 1 – point 3 – point b a (new)
          (b a)  AI systems intended to be used for the purpose of assessing the appropriate level of education for an individual and materially influencing the level of education and vocational training that individual will receive or will be able to access;
        Amendment 718
        Proposal for a regulation
        Annex III – paragraph 1 – point 3 – point b b (new)
          (b b)  AI systems intended to be used for monitoring and detecting prohibited behaviour of students during tests in the context of/within education and vocational training institutions;
        Amendment 719
        Proposal for a regulation
        Annex III – paragraph 1 – point 4 – point a
        (a)  AI systems intended to be used for recruitment or selection of natural persons, notably for advertising vacancies, screening or filtering applications, evaluating candidates in the course of interviews or tests;
          (a)  AI systems intended to be used for recruitment or selection of natural persons, notably for placing targeted job advertisements screening or filtering applications, evaluating candidates in the course of interviews or tests;
        Amendment 720
        Proposal for a regulation
        Annex III – paragraph 1 – point 4 – point b
        (b)  AI intended to be used for making decisions on promotion and termination of work-related contractual relationships, for task allocation and for monitoring and evaluating performance and behavior of persons in such relationships.
          (b)  AI systems intended to be used to make or materially influence decisions affecting the initiation, promotion and termination of work-related contractual relationships, task allocation based on individual behaviour or personal traits or characteristics, or for monitoring and evaluating performance and behavior of persons in such relationships;
        Amendment 721
        Proposal for a regulation
        Annex III – paragraph 1 – point 5 – point a
        (a)  AI systems intended to be used by public authorities or on behalf of public authorities to evaluate the eligibility of natural persons for public assistance benefits and services, as well as to grant, reduce, revoke, or reclaim such benefits and services;
          (a)  AI systems intended to be used by or on behalf of public authorities to evaluate the eligibility of natural persons for public assistance benefits and services, including healthcare services and essential services, including but not limited to housing, electricity, heating/cooling and internet, as well as to grant, reduce, revoke, increase or reclaim such benefits and services;
        Amendment 722
        Proposal for a regulation
        Annex III – paragraph 1 – point 5 – point b
        (b)  AI systems intended to be used to evaluate the creditworthiness of natural persons or establish their credit score, with the exception of AI systems put into service by small scale providers for their own use;
          (b)  AI systems intended to be used to evaluate the creditworthiness of natural persons or establish their credit score , with the exception of AI systems used for the purpose of detecting financial fraud;
        Amendment 723
        Proposal for a regulation
        Annex III – paragraph 1 – point 5 – point b a (new)
          (b a)  AI systems intended to be used for making decisions or materially influencing decisions on the eligibility of natural persons for health and life insurance;
        Amendment 724
        Proposal for a regulation
        Annex III – paragraph 1 – point 5 – point c
        (c)  AI systems intended to be used to dispatch, or to establish priority in the dispatching of emergency first response services, including by firefighters and medical aid.
          (c)  AI systems intended to evaluate and classify emergency calls by natural persons or to be used to dispatch, or to establish priority in the dispatching of emergency first response services, including by police and law enforcement, firefighters and medical aid, as well as of emergency healthcare patient triage systems;
        Amendment 725
        Proposal for a regulation
        Annex III – paragraph 1 – point 6 – point a
        (a)  AI systems intended to be used by law enforcement authorities for making individual risk assessments of natural persons in order to assess the risk of a natural person for offending or reoffending or the risk for potential victims of criminal offences;
          deleted
        Amendment 726
        Proposal for a regulation
        Annex III – paragraph 1 – point 6 – point b
        (b)  AI systems intended to be used by law enforcement authorities as polygraphs and similar tools or to detect the emotional state of a natural person;
          (b)  AI systems intended to be used by or on behalf of law enforcement authorities, or by Union agencies, offices or bodies in support of law enforcement authorities as polygraphs and similar tools, insofar as their use is permitted under relevant Union and national law;
        Amendment 727
        Proposal for a regulation
        Annex III – paragraph 1 – point 6 – point c
        (c)  AI systems intended to be used by law enforcement authorities to detect deep fakes as referred to in article 52(3);
          deleted
        Amendment 728
        Proposal for a regulation
        Annex III – paragraph 1 – point 6 – point d
        (d)  AI systems intended to be used by law enforcement authorities for evaluation of the reliability of evidence in the course of investigation or prosecution of criminal offences;
          (d)  AI systems intended to be used by or on behalf of law enforcement authorities, or by Union agencies, offices or bodies in support of law enforcement authorities to evaluate the reliability of evidence in the course of investigation or prosecution of criminal offences;
        Amendment 729
        Proposal for a regulation
        Annex III – paragraph 1 – point 6 – point e
        (e)  AI systems intended to be used by law enforcement authorities for predicting the occurrence or reoccurrence of an actual or potential criminal offence based on profiling of natural persons as referred to in Article 3(4) of Directive (EU) 2016/680 or assessing personality traits and characteristics or past criminal behaviour of natural persons or groups;
          deleted
        Amendment 730
        Proposal for a regulation
        Annex III – paragraph 1 – point 6 – point f
        (f)  AI systems intended to be used by law enforcement authorities for profiling of natural persons as referred to in Article 3(4) of Directive (EU) 2016/680 in the course of detection, investigation or prosecution of criminal offences;
          (f)  AI systems intended to be used by or on behalf of law enforcement authorities or by Union agencies, offices or bodies in support of law enforcement authorities for profiling of natural persons as referred to in Article 3(4) of Directive (EU) 2016/680 in the course of detection, investigation or prosecution of criminal offences or, in the case of Union agencies, offices or bodies, as referred to in Article 3(5) of Regulation (EU) 2018/1725;
        Amendment 731
        Proposal for a regulation
        Annex III – paragraph 1 – point 6 – point g
        (g)  AI systems intended to be used for crime analytics regarding natural persons, allowing law enforcement authorities to search complex related and unrelated large data sets available in different data sources or in different data formats in order to identify unknown patterns or discover hidden relationships in the data.
          (g)  AI systems intended to be used by or on behalf of law enforcement authorities or by Union agencies, offices or bodies in support of law enforcement authorities for crime analytics regarding natural persons, allowing law enforcement authorities to search complex related and unrelated large data sets available in different data sources or in different data formats in order to identify unknown patterns or discover hidden relationships in the data.
        Amendment 732
        Proposal for a regulation
        Annex III – paragraph 1 – point 7 – point a
        (a)  AI systems intended to be used by competent public authorities as polygraphs and similar tools or to detect the emotional state of a natural person;
          (a)  AI systems intended to be used by or on behalf of competent public authorities or by Union agencies, offices or bodies as polygraphs and similar tools insofar as their use is permitted under relevant Union or national law
        Amendment 733
        Proposal for a regulation
        Annex III – paragraph 1 – point 7 – point b
        (b)  AI systems intended to be used by competent public authorities to assess a risk, including a security risk, a risk of irregular immigration, or a health risk, posed by a natural person who intends to enter or has entered into the territory of a Member State;
          (b)  AI systems intended to be used by or on behalf of competent public authorities or by Union agencies, offices or bodies to assess a risk, including a security risk, a risk of irregular immigration, or a health risk, posed by a natural person who intends to enter or has entered into the territory of a Member State;
        Amendment 734
        Proposal for a regulation
        Annex III – paragraph 1 – point 7 – point c
        (c)  AI systems intended to be used by competent public authorities for the verification of the authenticity of travel documents and supporting documentation of natural persons and detect non-authentic documents by checking their security features;
          (c)  AI systems intended to be used by or on behalf of competent public authorities or by Union agencies, offices or bodies for the verification of the authenticity of travel documents and supporting documentation of natural persons and detect non-authentic documents by checking their security features;
        Amendment 735
        Proposal for a regulation
        Annex III – paragraph 1 – point 7 – point d
        (d)  AI systems intended to assist competent public authorities for the examination of applications for asylum, visa and residence permits and associated complaints with regard to the eligibility of the natural persons applying for a status.
          (d)  AI systems intended to be used by or on behalf of competent public authorities or by Union agencies, offices or bodies to assist competent public authorities for the examination and assessment of the veracity of evidence in relation to applications for asylum, visa and residence permits and associated complaints with regard to the eligibility of the natural persons applying for a status;
        Amendment 736
        Proposal for a regulation
        Annex III – paragraph 1 – point 7 – point d a (new)
          (d a)  AI systems intended to be used by or on behalf of competent public authorities or by Union agencies, offices or bodies in migration, asylum and border control management to monitor, surveil or process data in the context of border management activities, for the purpose of detecting, recognising or identifying natural persons;
        Amendment 737
        Proposal for a regulation
        Annex III – paragraph 1 – point 7 – point d b (new)
          (d b)  AI systems intended to be used by or on behalf of competent public authorities or by Union agencies, offices or bodies in migration, asylum and border control management for the forecasting or prediction of trends related to migration movement and border crossing;
        Amendment 738
        Proposal for a regulation
        Annex III – paragraph 1 – point 8 – point a
        (a)  AI systems intended to assist a judicial authority in researching and interpreting facts and the law and in applying the law to a concrete set of facts.
          (a)  AI systems intended to be used by a judicial authority ot administrative body or on their behalf to assist a judicial authority or administrative body in researching and interpreting facts and the law and in applying the law to a concrete set of facts or used in a similar way in alternative dispute resolution.
        Amendment 739
        Proposal for a regulation
        Annex III – paragraph 1 – point 8 – point a a (new)
          (a a)  AI systems intended to be used for influencing the outcome of an election or referendum or the voting behaviour of natural persons in the exercise of their vote in elections or referenda. This does not include AI systems whose output natural persons are not directly exposed to, such as tools used to organise, optimise and structure political campaigns from an administrative and logistic point of view.
        Amendment 740
        Proposal for a regulation
        Annex III – paragraph 1 – point 8 – point a b (new)
          (a b)  AI systems intended to be used by social media platforms that have been designated as very large online platforms within the meaning of Article 33 of Regulation EU 2022/2065, in their recommender systems to recommend to the recipient of the service user-generated content available on the platform.

        Here are the details of the AI system:

        Domain: "{}",
        Purpose: "{}",
        Capability: "{}",
        AI User: "{}",
        AI Subject: "{}",
        Description: "{}",
        Classification: "{}",
        Classification_GPT: "{}",
        Reasoning: "{}",
        AIActText: "{}"

         Please return the mitigation strategy in the following format:
         {{
           "Mitigation Strategy": "The AI system can become low risk if ...",
           "Type of Mitigation Strategy": "[Avoidance/Reduction/Transference/Acceptance]",
           "New System Description": "The mitigated AI system is intended to be used ...",
           "New Classification": ["Unacceptable Risk"/"High Risk"/"Not Classified as High Risk or Unacceptable Risk"],
           "Relevant Text from the EU AI Act": "[Quotation if applicable] - Include the amendment or EU AI Act section that mostly closely resembles the text.",
           "Reasoning": "[Explanation]"
         }}
            """
    }
]



def format_prompt(MESSAGES, domain,purpose,aiCapability,aiUser,aiSubject,description,classification,classification_GPT, reasoning,AIActText):
    S = "test {}"
    messages = deepcopy(MESSAGES)
    messages[1]['content'] = messages[1]['content'].format(domain,purpose,aiCapability,aiUser,aiSubject,description,classification,classification_GPT, reasoning,AIActText)
    return messages


## 1 APPLY / RUN

In [ ]:
EUAIACT_risks

In [ ]:
cost = 0

FULL_RES = []

start_time = time.time()
i = 0

for useElements in EUAIACT_risks:
  useI = str(useElements['id'])

  print (f" Parsing use {useI}")

  # Variables for message placeholders
  domain = useElements['Details'][0]
  purpose = useElements['Details'][1]
  aiCapability = useElements['Details'][2]
  aiUser = useElements['Details'][3]
  aiSubject = useElements['Details'][4]

  description = useElements['Description']
  classification = useElements['Classification_Annotators']
  classification_GPT = useElements['Classification_GPT']
  reasoning = useElements['Reasoning']
  AIActText = useElements['AIActText']

  # Extracting "Use i" details
  use_i_details = [domain, purpose, aiCapability, aiUser, aiSubject, description, classification, classification_GPT, reasoning, AIActText]

  print(use_i_details)

  # adapt the prompt for useI
  messages = format_prompt(MESSAGES, domain, purpose, aiCapability, aiUser, aiSubject, description, classification, classification_GPT, reasoning, AIActText)

  # run the prompt
  # response = get_completion_from_messages(messages, temperature=0)
  # print(response)

  response, token_count = get_completion_and_token_count(messages, temperature=0)
  res = token_count
  cost_chunk = (res['prompt_tokens'] * 0.03  + res['completion_tokens'] * 0.06)/1000.0
  cost += cost_chunk

  response = ast.literal_eval(response)

  print(replace_key(response, "Relevant Text from the EU AI Act", "AIActText"))

  combined_response = {}
  combined_response["id"]= useI
  combined_response["Details"] = use_i_details
  for k, v in response.items():
    combined_response[k] = v
  print (combined_response)


  ###############################
  # save result
  # with open(f"EU_AI_Act_risks_mitigation.json", "w") as json_file:
  #     json.dump(combined_response, json_file, indent=4)  # 4 spaces of indentation
  # # Download the file to your local machine
  # files.download(f"EU_AI_Act_risks_mitigation.json")
  end_time = time.time()

  print(f"Execution time: {end_time - start_time:.5f} seconds")
  print (f"TOTAL COST {cost}")

  FULL_RES.append(combined_response)

 Parsing use 15
['Well-being', 'Personal growth coaching', 'Analyzing facial responses to personal growth exercises', 'Personal growth apps, coaches', 'Coaching clients', 'The AI system intended to be used for personal growth coaching, with the capability of analyzing facial responses to personal growth exercises. The users of this AI system are personal growth apps and coaches, and the subjects are coaching clients.', 'Not Classified as High Risk or Unacceptable Risk', 'High Risk', 'The AI system in question is used for personal growth coaching and analyzes facial responses, which can be considered a form of biometric data. According to the EU AI Act and its amendments, AI systems that use biometric data for identification or to make inferences about personal characteristics are classified as high risk. Therefore, this AI system falls under the high-risk category.', 'High-risk AI systems pursuant to Article 6(2) are the AI systems listed in any of the following areas: 1. Biometric ide

In [ ]:
###############################
# save result
with open(f"mitigations_AI_subjects_7s.json", "w") as json_file:
    json.dump(FULL_RES, json_file, indent=4)  # 4 spaces of indentation
# Download the file to your local machine
files.download(f"mitigations_AI_subjects_7s.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 2 SDG Risks MITIGATION PROMPT

In [ ]:
# Assuming you have the variables domain, purpose, aiCapability, aiUser, and aiSubject defined with appropriate values

MESSAGES = [
    {
        'role': 'system',
        'content': """You are a renowned specialist in the field of AI technology with a dedicated focus on understanding, promoting, and implementing the Sustainable Development Goals (SDGs). With your vast experience, decisiveness, and conscientious approach, you have a deep understanding of how AI technologies can be leveraged to support the SDGs.
        You possess comprehensive knowledge of the List of Sustainable Development Goal targets and indicators, which encapsulates all targets and indicators for the 17 SDGs. This global framework was meticulously crafted by the Inter-Agency and Expert Group on SDG Indicators (IAEG-SDGs) and was ratified during the 48th session of the United Nations Statistical Commission in March 2017.
        The framework you reference encompasses all subsequent refinements to the official indicator list."""
    },
    {
        'role': 'user',
        'content': """

        The AI System provided in input is assessed as potentially undermining the developmet of specific SDGs. For each SDG, consider carefully the system's output, as well as the reasoning, and propose a mitigation strategy that would make the system not to anymore undermine the specific SDGs. Propose a mitigation strategy for each undermined goal.
        Moreover, label the type of the mitigation strategy proposed considering the definitions of 4 possible labels below:
          Avoidance: take measures to avoid the risk from occurring.
          Reduction: reduce the likelihood of a risk happening or the impact should it occur.
          Transference: pass the risk consequence to a third party.
          Acceptance: accepting the risk as it stands (this means, no mitigation strategy was found).

        This assessment requires a deep understanding of both the AI system in question and the definitions of the Sustainable Development Goals. Each evaluation must be backed by solid reasoning and a clear understanding of the interplay between technology and SDG Goals. Remember, precision is paramount. Be succinct.

        Goal 1: End poverty in all its forms everywhere
        Targets:
        1.1 By 2030, eradicate extreme poverty for all people everywhere, currently measured as people living on less than $1.25 a day
        1.2 By 2030, reduce at least by half the proportion of men, women and children of all ages living in poverty in all its dimensions according to national definitions
        1.3 Implement nationally appropriate social protection systems and measures for all, including floors, and by 2030 achieve substantial coverage of the poor and the vulnerable
        1.4 By 2030, ensure that all men and women, in particular the poor and the vulnerable, have equal rights to economic resources, as well as access to basic services, ownership and control over land and other forms of property, inheritance, natural resources, appropriate new technology and financial services, including microfinance
        1.5 By 2030, build the resilience of the poor and those in vulnerable situations and reduce their exposure and vulnerability to climate-related extreme events and other economic, social and environmental shocks and disasters
        1.a Ensure significant mobilization of resources from a variety of sources, including through enhanced development cooperation, in order to provide adequate and predictable means for developing countries, in particular least developed countries, to implement programmes and policies to end poverty in all its dimensions
        1.b Create sound policy frameworks at the national, regional and international levels, based on pro-poor and gender-sensitive development strategies, to support accelerated investment in poverty eradication actions

        Indicators:
        1.1.1 Proportion of the population living below the international poverty line by sex, age, employment status and geographic location (urban/rural)
        1.2.1 Proportion of population living below the national poverty line, by sex and age
        1.2.2 Proportion of men, women and children of all ages living in poverty in all its dimensions according to national definitions
        1.3.1 Proportion of population covered by social protection floors/systems, by sex, distinguishing children, unemployed persons, older persons, persons with disabilities, pregnant women, newborns, work-injury victims and the poor and the vulnerable
        1.4.1 Proportion of population living in households with access to basic services
        1.4.2 Proportion of total adult population with secure tenure rights to land, (a) with legally recognized documentation, and (b) who perceive their rights to land as secure, by sex and type of tenure
        1.5.1 Number of deaths, missing persons and directly affected persons attributed to disasters per 100,000 population
        1.5.2 Direct economic loss attributed to disasters in relation to global gross domestic product (GDP)
        1.5.3 Number of countries that adopt and implement national disaster risk reduction strategies in line with the Sendai Framework for Disaster Risk Reduction 2015–2030
        1.5.4 Proportion of local governments that adopt and implement local disaster risk reduction strategies in line with national disaster risk reduction strategies
        1.a.1 Total official development assistance grants from all donors that focus on poverty reduction as a share of the recipient country's gross national income
        1.a.2 Proportion of total government spending on essential services (education, health and social protection)
        1.b.1 Pro-poor public social spending


        Goal 2: End hunger, achieve food security and improved nutrition and promote sustainable agriculture
        Targets:
        2.1 By 2030, end hunger and ensure access by all people, in particular the poor and people in vulnerable situations, including infants, to safe, nutritious and sufficient food all year round.
        2.2 By 2030, end all forms of malnutrition, including achieving, by 2025, the internationally agreed targets on stunting and wasting in children under 5 years of age, and address the nutritional needs of adolescent girls, pregnant and lactating women and older persons.
        2.3 By 2030, double the agricultural productivity and incomes of small-scale food producers, in particular women, indigenous peoples, family farmers, pastoralists and fishers, including through secure and equal access to land, other productive resources and inputs, knowledge, financial services, markets and opportunities for value addition and non-farm employment.
        2.4 By 2030, ensure sustainable food production systems and implement resilient agricultural practices that increase productivity and production, that help maintain ecosystems, that strengthen capacity for adaptation to climate change, extreme weather, drought, flooding and other disasters and that progressively improve land and soil quality.
        2.5 By 2020, maintain the genetic diversity of seeds, cultivated plants and farmed and domesticated animals and their related wild species, including through soundly managed and diversified seed and plant banks at the national, regional and international levels, and promote access to and fair and equitable sharing of benefits arising from the utilization of genetic resources and associated traditional knowledge, as internationally agreed.
        2.a Increase investment, including through enhanced international cooperation, in rural infrastructure, agricultural research and extension services, technology development and plant and livestock gene banks in order to enhance agricultural productive capacity in developing countries, in particular least developed countries.
        2.b Correct and prevent trade restrictions and distortions in world agricultural markets, including through the parallel elimination of all forms of agricultural export subsidies and all export measures with equivalent effect, in accordance with the mandate of the Doha Development Round.
        2.c Adopt measures to ensure the proper functioning of food commodity markets and their derivatives and facilitate timely access to market information, including on food reserves, in order to help limit extreme food price volatility.

        Indicators:
        2.1.1 Prevalence of undernourishment.
        2.1.2 Prevalence of moderate or severe food insecurity in the population, based on the Food Insecurity Experience Scale (FIES).
        2.2.1 Prevalence of stunting (height for age <-2 standard deviation from the median of the World Health Organization (WHO) Child Growth Standards) among children under 5 years of age.
        2.2.2 Prevalence of malnutrition (weight for height >+2 or <-2 standard deviation from the median of the WHO Child Growth Standards) among children under 5 years of age, by type (wasting and overweight).
        2.2.3 Prevalence of anaemia in women aged 15 to 49 years, by pregnancy status (percentage).
        2.3.1 Volume of production per labour unit by classes of farming/pastoral/forestry enterprise size.
        2.3.2 Average income of small-scale food producers, by sex and indigenous status.
        2.4.1 Proportion of agricultural area under productive and sustainable agriculture.
        2.5.1 Number of plant and animal genetic resources for food and agriculture secured in either medium- or long-term conservation facilities.
        2.5.2 Proportion of local breeds classified as being at risk of extinction.
        2.a.1 The agriculture orientation index for government expenditures.
        2.a.2 Total official flows (official development assistance plus other official flows) to the agriculture sector.
        2.b.1 Agricultural export subsidies.
        2.c.1 Indicator of food price anomalies.

        Goal 3: Ensure healthy lives and promote well-being for all at all ages
        Targets:
        3.1 By 2030, reduce the global maternal mortality ratio to less than 70 per 100,000 live births
        3.2 By 2030, end preventable deaths of newborns and children under 5 years of age, with all countries aiming to reduce neonatal mortality to at least as low as 12 per 1,000 live births and under‑5 mortality to at least as low as 25 per 1,000 live births
        3.3 By 2030, end the epidemics of AIDS, tuberculosis, malaria and neglected tropical diseases and combat hepatitis, water-borne diseases and other communicable diseases
        3.4 By 2030, reduce by one third premature mortality from non-communicable diseases through prevention and treatment and promote mental health and well-being
        3.5 Strengthen the prevention and treatment of substance abuse, including narcotic drug abuse and harmful use of alcohol
        3.6 By 2020, halve the number of global deaths and injuries from road traffic accidents
        3.7 By 2030, ensure universal access to sexual and reproductive health-care services, including for family planning, information and education, and the integration of reproductive health into national strategies and programmes
        3.8 Achieve universal health coverage, including financial risk protection, access to quality essential health-care services and access to safe, effective, quality and affordable essential medicines and vaccines for all
        3.9 By 2030, substantially reduce the number of deaths and illnesses from hazardous chemicals and air, water and soil pollution and contamination
        3.a Strengthen the implementation of the World Health Organization Framework Convention on Tobacco Control in all countries, as appropriate
        3.b Support the research and development of vaccines and medicines for the communicable and non‑communicable diseases that primarily affect developing countries, provide access to affordable essential medicines and vaccines
        3.c Substantially increase health financing and the recruitment, development, training and retention of the health workforce in developing countries, especially in least developed countries and small island developing States
        3.d Strengthen the capacity of all countries, in particular developing countries, for early warning, risk reduction and management of national and global health risks

        Indicators:
        3.1.1 Maternal mortality ratio
        3.1.2 Proportion of births attended by skilled health personnel
        3.2.1 Under‑5 mortality rate
        3.2.2 Neonatal mortality rate
        3.3.1 Number of new HIV infections per 1,000 uninfected population, by sex, age and key populations
        3.3.2 Tuberculosis incidence per 100,000 population
        3.3.3 Malaria incidence per 1,000 population
        3.3.4 Hepatitis B incidence per 100,000 population
        3.3.5 Number of people requiring interventions against neglected tropical diseases
        3.4.1 Mortality rate attributed to cardiovascular disease, cancer, diabetes or chronic respiratory disease
        3.4.2 Suicide mortality rate
        3.5.1 Coverage of treatment interventions (pharmacological, psychosocial and rehabilitation and aftercare services) for substance use disorders
        3.5.2 Alcohol per capita consumption (aged 15 years and older) within a calendar year in litres of pure alcohol
        3.6.1 Death rate due to road traffic injuries
        3.7.1 Proportion of women of reproductive age (aged 15–49 years) who have their need for family planning satisfied with modern methods
        3.7.2 Adolescent birth rate (aged 10–14 years; aged 15–19 years) per 1,000 women in that age group
        3.8.1 Coverage of essential health services
        3.8.2 Proportion of population with large household expenditures on health as a share of total household expenditure or income
        3.9.1 Mortality rate attributed to household and ambient air pollution
        3.9.2 Mortality rate attributed to unsafe water, unsafe sanitation and lack of hygiene (exposure to unsafe Water, Sanitation and Hygiene for All (WASH) services)
        3.9.3 Mortality rate attributed to unintentional poisoning
        3.a.1 Age-standardized prevalence of current tobacco use among persons aged 15 years and older
        3.b.1 Proportion of the target population covered by all vaccines included in their national programme
        3.b.2 Total net official development assistance to medical research and basic health sectors
        3.b.3 Proportion of health facilities that have a core set of relevant essential medicines available and affordable on a sustainable basis
        3.c.1 Health worker density and distribution
        3.d.1 International Health Regulations (IHR) capacity and health emergency preparedness
        3.d.2 Percentage of bloodstream infections due to selected antimicrobial-resistant organisms


        Goal 4: Ensure inclusive and equitable quality education and promote lifelong learning opportunities for all
        Targets:
        4.1 By 2030, ensure that all girls and boys complete free, equitable and quality primary and secondary education leading to relevant and effective learning outcomes
        4.2 By 2030, ensure that all girls and boys have access to quality early childhood development, care and pre‑primary education so that they are ready for primary education
        4.3 By 2030, ensure equal access for all women and men to affordable and quality technical, vocational and tertiary education, including university
        4.4 By 2030, substantially increase the number of youth and adults who have relevant skills, including technical and vocational skills, for employment, decent jobs and entrepreneurship
        4.5 By 2030, eliminate gender disparities in education and ensure equal access to all levels of education and vocational training for the vulnerable, including persons with disabilities, indigenous peoples and children in vulnerable situations
        4.6 By 2030, ensure that all youth and a substantial proportion of adults, both men and women, achieve literacy and numeracy
        4.7 By 2030, ensure that all learners acquire the knowledge and skills needed to promote sustainable development, including, among others, through education for sustainable development and sustainable lifestyles, human rights, gender equality, promotion of a culture of peace and non-violence, global citizenship and appreciation of cultural diversity and of culture's contribution to sustainable development
        4.a Build and upgrade education facilities that are child, disability and gender sensitive and provide safe, non-violent, inclusive and effective learning environments for all
        4.b By 2020, substantially expand globally the number of scholarships available to developing countries, in particular least developed countries, small island developing States and African countries, for enrolment in higher education, including vocational training and information and communications technology, technical, engineering and scientific programmes, in developed countries and other developing countries
        4.c By 2030, substantially increase the supply of qualified teachers, including through international cooperation for teacher training in developing countries, especially least developed countries and small island developing States

        Indicators:
        4.1.1 Proportion of children and young people (a) in grades 2/3; (b) at the end of primary; and (c) at the end of lower secondary achieving at least a minimum proficiency level in (i) reading and (ii) mathematics, by sex
        4.1.2 Completion rate (primary education, lower secondary education, upper secondary education)
        4.2.1 Proportion of children aged 24–59 months who are developmentally on track in health, learning and psychosocial well-being, by sex
        4.2.2 Participation rate in organized learning (one year before the official primary entry age), by sex
        4.3.1 Participation rate of youth and adults in formal and non-formal education and training in the previous 12 months, by sex
        4.4.1 Proportion of youth and adults with information and communications technology (ICT) skills, by type of skill
        4.5.1 Parity indices (female/male, rural/urban, bottom/top wealth quintile and others such as disability status, indigenous peoples and conflict-affected, as data become available) for all education indicators on this list that can be disaggregated
        4.6.1 Proportion of population in a given age group achieving at least a fixed level of proficiency in functional (a) literacy and (b) numeracy skills, by sex
        4.7.1 Extent to which (i) global citizenship education and (ii) education for sustainable development are mainstreamed in (a) national education policies; (b) curricula; (c) teacher education; and (d) student assessment
        4.a.1 Proportion of schools offering basic services, by type of service
        4.b.1 Volume of official development assistance flows for scholarships by sector and type of study
        4.c.1 Proportion of teachers with the minimum required qualifications, by education level

        Goal 5: Achieve gender equality and empower all women and girls
        Targets:
        5.1 End all forms of discrimination against all women and girls everywhere
        5.2 Eliminate all forms of violence against all women and girls in the public and private spheres, including trafficking and sexual and other types of exploitation
        5.3 Eliminate all harmful practices, such as child, early and forced marriage and female genital mutilation
        5.4 Recognize and value unpaid care and domestic work through the provision of public services, infrastructure and social protection policies and the promotion of shared responsibility within the household and the family as nationally appropriate
        5.5 Ensure women's full and effective participation and equal opportunities for leadership at all levels of decision-making in political, economic and public life
        5.6 Ensure universal access to sexual and reproductive health and reproductive rights as agreed in accordance with the Programme of Action of the International Conference on Population and Development and the Beijing Platform for Action and the outcome documents of their review conferences
        5.a Undertake reforms to give women equal rights to economic resources, as well as access to ownership and control over land and other forms of property, financial services, inheritance and natural resources, in accordance with national laws
        5.b Enhance the use of enabling technology, in particular information and communications technology, to promote the empowerment of women
        5.c Adopt and strengthen sound policies and enforceable legislation for the promotion of gender equality and the empowerment of all women and girls at all levels

        Indicators:
        5.1.1 Whether or not legal frameworks are in place to promote, enforce and monitor equality and non‑discrimination on the basis of sex
        5.2.1 Proportion of ever-partnered women and girls aged 15 years and older subjected to physical, sexual or psychological violence by a current or former intimate partner in the previous 12 months, by form of violence and by age
        5.2.2 Proportion of women and girls aged 15 years and older subjected to sexual violence by persons other than an intimate partner in the previous 12 months, by age and place of occurrence
        5.3.1 Proportion of women aged 20–24 years who were married or in a union before age 15 and before age 18
        5.3.2 Proportion of girls and women aged 15–49 years who have undergone female genital mutilation/cutting, by age
        5.4.1 Proportion of time spent on unpaid domestic and care work, by sex, age and location
        5.5.1 Proportion of seats held by women in (a) national parliaments and (b) local governments
        5.5.2 Proportion of women in managerial positions
        5.6.1 Proportion of women aged 15–49 years who make their own informed decisions regarding sexual relations, contraceptive use and reproductive health care
        5.6.2 Number of countries with laws and regulations that guarantee full and equal access to women and men aged 15 years and older to sexual and reproductive health care, information and education
        5.a.1 (a) Proportion of total agricultural population with ownership or secure rights over agricultural land, by sex; and (b) share of women among owners or rights-bearers of agricultural land, by type of tenure
        5.a.2 Proportion of countries where the legal framework (including customary law) guarantees women's equal rights to land ownership and/or control
        5.b.1 Proportion of individuals who own a mobile telephone, by sex
        5.c.1 Proportion of countries with systems to track and make public allocations for gender equality and women's empowerment

        Here are the details of the AI system:

        Domain: "{}",
        Purpose: "{}",
        Capability: "{}",
        AI User: "{}",
        AI Subject: "{}",
        Description: "{}",
        SDG Assessment: "{}"

         Please return the mitigation strategies in the following format, and MAKE SURE TO OUTPUT CORRECTLY FORMATTED JSON:
         {{
           "Description": "The AI system intended to be used ...",
           "New Assessment of impact on human rights"":[
           {{
           "Mitigation Strategy for [SDG Number1]" : "The AI system will not undermine this SDG goal if ...",
           "Type of Mitigation Strategy for [SDG Number1]": "[Avoidance/Reduction/Transference/Acceptance]",
           "New SDG [SDG Number1] Assessment" : "[Positive/Negative/Mixed/Inapplicable]",
           "Reasoning for NOT anymore Undermining SDG [SDG Number1]": "[Explanation]"
           }},
           {{
           "Mitigation Strategy for [SDG Number2]" : "The AI system will not undermine this SDG goal if ...",
           "Type of Mitigation Strategy for [SDG Number2]": "[Avoidance/Reduction/Transference/Acceptance]",
           "New SDG [SDG Number2] Assessment" : "[Positive/Negative/Mixed/Inapplicable]",
           "Reasoning for NOT anymore Undermining SDG [SDG Number2]": "[Explanation]"
           }},
           ...
           {{
           "Mitigation Strategy for [SDG NumberN]" : "The AI system will not undermine this SDG goal if ...",
           "Type of Mitigation Strategy for [SDG NumberN]": "[Avoidance/Reduction/Transference/Acceptance]",
           "New SDG [SDG NumberN] Assessment" : "[Positive/Negative/Mixed/Inapplicable]",
           "Reasoning for NOT anymore Undermining SDG [SDG NumberN]": "[Explanation]"
           }},
           ]
        }}
            """
    }
]



def format_prompt(MESSAGES, domain,purpose,aiCapability,aiUser,aiSubject,description,SDGAssessment):
    S = "test {}"
    messages = deepcopy(MESSAGES)
    messages[1]['content'] = messages[1]['content'].format(domain,purpose,aiCapability,aiUser,aiSubject,description,SDGAssessment)
    return messages


## 2 APPLY / RUN

In [ ]:
cost = 0

start_time = time.time()
i = 0

useElements = SDG_risks
useI = str(useElements['id'])

print (f" Parsing use {useI}")

# Variables for message placeholders
domain = useElements['Details'][0]
purpose = useElements['Details'][1]
aiCapability = useElements['Details'][2]
aiUser = useElements['Details'][3]
aiSubject = useElements['Details'][4]

description = useElements['Description']
SDGAssessment = useElements['SDG Assessment']

# Extracting "Use i" details
use_i_details = [domain, purpose, aiCapability, aiUser, aiSubject, description, SDGAssessment]

print(use_i_details)

# adapt the prompt for useI
messages = format_prompt(MESSAGES, domain, purpose, aiCapability, aiUser, aiSubject, description, SDGAssessment)

# run the prompt
# response = get_completion_from_messages(messages, temperature=0)
# print(response)

response, token_count = get_completion_and_token_count(messages, temperature=0)
res = token_count
cost_chunk = (res['prompt_tokens'] * 0.03  + res['completion_tokens'] * 0.06)/1000.0
cost += cost_chunk

response = ast.literal_eval(response)

print(replace_key(response, "Relevant Text from the EU AI Act", "AIActText"))

combined_response = {}
combined_response["id"]= useI
combined_response["Details"] = use_i_details
for k, v in response.items():
  combined_response[k] = v
print (combined_response)


###############################
# save result
with open(f"SDG_risks_mitigation.json", "w") as json_file:
    json.dump(combined_response, json_file, indent=4)  # 4 spaces of indentation
# Download the file to your local machine
files.download(f"SDG_risks_mitigation.json")
end_time = time.time()

print(f"Execution time: {end_time - start_time:.5f} seconds")
print (f"TOTAL COST {cost}")

 Parsing use 64
['Law enforcement', 'Crime prevention', 'Identify potential crime hotspots', 'CCTV', 'Police departments, security agencies', "The AI system is designed to identify potential crime hotspots for law enforcement agencies, using data from CCTV cameras. The system's purpose is to aid in crime prevention and enhance public safety.", [{'SDG 5': 'Mixed', 'Targets Supported for SDG 5': '5.2', 'Indicators Supported for SDG 5': '5.2.1, 5.2.2', 'Reasoning for Support for SDG 5': 'The AI system can help to eliminate violence against women and girls by identifying potential crime hotspots and enabling law enforcement to take preventive measures.', 'Targets Undermined for SDG 5': '5.b', 'Indicators Undermined for SDG 5': '5.b.1', 'Reasoning for Undermining for SDG 5': 'The AI system could potentially undermine gender equality if it is not designed and used in a gender-sensitive manner. For instance, if the system is biased in its identification of crime hotspots, it could lead to ove

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Execution time: 34.04029 seconds
TOTAL COST 0.18630000000000002


# 3 Human Rights MITIGATION PROMPT

In [ ]:
# Assuming you have the variables domain, purpose, aiCapability, aiUser, and aiSubject defined with appropriate values

MESSAGES = [
    {
        'role': 'system',
        'content': """You are a renowned specialist in the field of Employment, workers management and access to self-employment technology with a dedicated focus on understanding, promoting, and supporting the Universal Human Rights. With your vast experience, decisiveness, and conscientious approach, you have a deep understanding of how Employment, workers management and access to self-employment technologies can be leveraged to impact the Human Rights.
        You possess comprehensive knowledge of the List of human rights definitions, which encapsulates all 30 articles from the UN Universal Decalartion on Human Rights. This global framework was meticulously crafted by the United Nations.
        """
    },
    {
        'role': 'user',
        'content': """

        The AI System provided in input is assessed as potentially undermining the developmet of specific human rights. For each human right, consider carefully the system's output, as well as the reasoning, and propose a mitigation strategy that would make the system not to anymore undermine the specific human rights. Propose a mitigation strategy for each undermined goal.
        Moreover, label the type of the mitigation strategy proposed considering the definitions of 4 possible labels below:
          Avoidance: take measures to avoid the risk from occurring.
          Reduction: reduce the likelihood of a risk happening or the impact should it occur.
          Transference: pass the risk consequence to a third party.
          Acceptance: accepting the risk as it stands (this means, no mitigation strategy was found).

        This assessment requires a deep understanding of both the AI system in question and the definitions of the Sustainable Development Goals. Each evaluation must be backed by solid reasoning and a clear understanding of the interplay between technology and SDG Goals. Remember, precision is paramount. Be succinct.


        *** Universal Declaration of Human Rights ***

        Preamble
        Whereas recognition of the inherent dignity and of the equal and inalienable rights of all members of the human family is the foundation of freedom, justice and peace in the world,
        Whereas disregard and contempt for human rights have resulted in barbarous acts which have outraged the conscience of mankind, and the advent of a world in which human beings shall enjoy freedom of speech and belief and freedom from fear and want has been proclaimed as the highest aspiration of the common people,
        Whereas it is essential, if man is not to be compelled to have recourse, as a last resort, to rebellion against tyranny and oppression, that human rights should be protected by the rule of law,
        Whereas it is essential to promote the development of friendly relations between nations,
        Whereas the peoples of the United Nations have in the Charter reaffirmed their faith in fundamental human rights, in the dignity and worth of the human person and in the equal rights of men and women and have determined to promote social progress and better standards of life in larger freedom,
        Whereas Member States have pledged themselves to achieve, in co-operation with the United Nations, the promotion of universal respect for and observance of human rights and fundamental freedoms,
        Whereas a common understanding of these rights and freedoms is of the greatest importance for the full realization of this pledge,

        Now, therefore,

        The General Assembly,

        Proclaims this Universal Declaration of Human Rights as a common standard of achievement for all peoples and all nations, to the end that every individual and every organ of society, keeping this Declaration constantly in mind, shall strive by teaching and education to promote respect for these rights and freedoms and by progressive measures, national and international, to secure their universal and effective recognition and observance, both among the peoples of Member States themselves and among the peoples of territories under their jurisdiction.

        Article 1
        All human beings are born free and equal in dignity and rights. They are endowed with reason and conscience and should act towards one another in a spirit of brotherhood.

        Article 2
        Everyone is entitled to all the rights and freedoms set forth in this Declaration, without distinction of any kind, such as race, colour, sex, language, religion, political or other opinion, national or social origin, property, birth or other status. Furthermore, no distinction shall be made on the basis of the political, jurisdictional or international status of the country or territory to which a person belongs, whether it be independent, trust, non-self-governing or under any other limitation of sovereignty.

        Article 3
        Everyone has the right to life, liberty and security of person.

        Article 4
        No one shall be held in slavery or servitude; slavery and the slave trade shall be prohibited in all their forms.

        Article 5
        No one shall be subjected to torture or to cruel, inhuman or degrading treatment or punishment.

        Article 6
        Everyone has the right to recognition everywhere as a person before the law.

        Article 7
        All are equal before the law and are entitled without any discrimination to equal protection of the law. All are entitled to equal protection against any discrimination in violation of this Declaration and against any incitement to such discrimination.

        Article 8
        Everyone has the right to an effective remedy by the competent national tribunals for acts violating the fundamental rights granted him by the constitution or by law.

        Article 9
        No one shall be subjected to arbitrary arrest, detention or exile.

        Article 10
        Everyone is entitled in full equality to a fair and public hearing by an independent and impartial tribunal, in the determination of his rights and obligations and of any criminal charge against him.

        Article 11
        Everyone charged with a penal offence has the right to be presumed innocent until proved guilty according to law in a public trial at which he has had all the guarantees necessary for his defence.
        No one shall be held guilty of any penal offence on account of any act or omission which did not constitute a penal offence, under national or international law, at the time when it was committed. Nor shall a heavier penalty be imposed than the one that was applicable at the time the penal offence was committed.
        Article 12
        No one shall be subjected to arbitrary interference with his privacy, family, home or correspondence, nor to attacks upon his honour and reputation. Everyone has the right to the protection of the law against such interference or attacks.

        Article 13
        Everyone has the right to freedom of movement and residence within the borders of each state.
        Everyone has the right to leave any country, including his own, and to return to his country.

        Article 14
        Everyone has the right to seek and to enjoy in other countries asylum from persecution.
        This right may not be invoked in the case of prosecutions genuinely arising from non-political crimes or from acts contrary to the purposes and principles of the United Nations.

        Article 15
        Everyone has the right to a nationality.
        No one shall be arbitrarily deprived of his nationality nor denied the right to change his nationality.

        Article 16
        Men and women of full age, without any limitation due to race, nationality or religion, have the right to marry and to found a family. They are entitled to equal rights as to marriage, during marriage and at its dissolution.
        Marriage shall be entered into only with the free and full consent of the intending spouses.
        The family is the natural and fundamental group unit of society and is entitled to protection by society and the State.

        Article 17
        Everyone has the right to own property alone as well as in association with others.
        No one shall be arbitrarily deprived of his property.

        Article 18
        Everyone has the right to freedom of thought, conscience and religion; this right includes freedom to change his religion or belief, and freedom, either alone or in community with others and in public or private, to manifest his religion or belief in teaching, practice, worship and observance.

        Article 19
        Everyone has the right to freedom of opinion and expression; this right includes freedom to hold opinions without interference and to seek, receive and impart information and ideas through any media and regardless of frontiers.

        Article 20
        Everyone has the right to freedom of peaceful assembly and association.
        No one may be compelled to belong to an association.

        Article 21
        Everyone has the right to take part in the government of his country, directly or through freely chosen representatives.
        Everyone has the right of equal access to public service in his country.
        The will of the people shall be the basis of the authority of government; this will shall be expressed in periodic and genuine elections which shall be by universal and equal suffrage and shall be held by secret vote or by equivalent free voting procedures.

        Article 22
        Everyone, as a member of society, has the right to social security and is entitled to realization, through national effort and international co-operation and in accordance with the organization and resources of each State, of the economic, social and cultural rights indispensable for his dignity and the free development of his personality.

        Article 23
        Everyone has the right to work, to free choice of employment, to just and favourable conditions of work and to protection against unemployment.
        Everyone, without any discrimination, has the right to equal pay for equal work.
        Everyone who works has the right to just and favourable remuneration ensuring for himself and his family an existence worthy of human dignity, and supplemented, if necessary, by other means of social protection.
        Everyone has the right to form and to join trade unions for the protection of his interests.

        Article 24
        Everyone has the right to rest and leisure, including reasonable limitation of working hours and periodic holidays with pay.

        Article 25
        Everyone has the right to a standard of living adequate for the health and well-being of himself and of his family, including food, clothing, housing and medical care and necessary social services, and the right to security in the event of unemployment, sickness, disability, widowhood, old age or other lack of livelihood in circumstances beyond his control.
        Motherhood and childhood are entitled to special care and assistance. All children, whether born in or out of wedlock, shall enjoy the same social protection.

        Article 26
        Everyone has the right to education. Education shall be free, at least in the elementary and fundamental stages. Elementary education shall be compulsory. Technical and professional education shall be made generally available and higher education shall be equally accessible to all on the basis of merit.
        Education shall be directed to the full development of the human personality and to the strengthening of respect for human rights and fundamental freedoms. It shall promote understanding, tolerance and friendship among all nations, racial or religious groups, and shall further the activities of the United Nations for the maintenance of peace.
        Parents have a prior right to choose the kind of education that shall be given to their children.

        Article 27
        Everyone has the right freely to participate in the cultural life of the community, to enjoy the arts and to share in scientific advancement and its benefits.
        Everyone has the right to the protection of the moral and material interests resulting from any scientific, literary or artistic production of which he is the author.

        Article 28
        Everyone is entitled to a social and international order in which the rights and freedoms set forth in this Declaration can be fully realized.

        Article 29
        Everyone has duties to the community in which alone the free and full development of his personality is possible.
        In the exercise of his rights and freedoms, everyone shall be subject only to such limitations as are determined by law solely for the purpose of securing due recognition and respect for the rights and freedoms of others and of meeting the just requirements of morality, public order and the general welfare in a democratic society.
        These rights and freedoms may in no case be exercised contrary to the purposes and principles of the United Nations.

        Article 30
        Nothing in this Declaration may be interpreted as implying for any State, group or person any right to engage in any activity or to perform any act aimed at the destruction of any of the rights and freedoms set forth herein.



        Here are the details of the AI system:

        Domain: "{}",
        Purpose: "{}",
        Capability: "{}",
        AI User: "{}",
        AI Subject: "{}",
        Description: "{}",
        Assessment of impact on human rights: "{}"


         In the input field "Assessment of impact on human rights", you will find the Articles that are potentially undermined by this use (or they are both promoted and undermined, i.e., the mixed label), along with explanations for that classification.
         Please return the mitigation strategies for these uses in the following format, where [ID1, ... ,IDn] will correspond to the Article IDs you find in input under "Assessment of impact on human rights" field, i.e., those that were previously found to be undermined by this use and that you want to mitigate,
         and MAKE SURE TO OUTPUT CORRECTLY FORMATTED JSON:
         {{
           "Description": "The AI system intended to be used ...",
           "New Assessment of impact on human rights"":[
           {{
           "Mitigation Strategy for Article ID1" : "The AI system will not undermine this human right if ...",
           "Type of Mitigation Strategy for Article ID1": "[Avoidance/Reduction/Transference/Acceptance]",
           "New Article ID1 Assessment" : "[Positive/Negative/Mixed/Inapplicable]",
           "Reasoning for NOT anymore Undermining Article ID1": "[Explanation]"
           }},
           ...
           {{
           "Mitigation Strategy for Article IDn" : "The AI system will not undermine this human right if ...",
           "Type of Mitigation Strategy for Article IDn": "[Avoidance/Reduction/Transference/Acceptance]",
           "New Article IDn Assessment" : "[Positive/Negative/Mixed/Inapplicable]",
           "Reasoning for NOT anymore Undermining Article IDn": "[Explanation]"
           }}
           ]
        }}
        """
    }
]



def format_prompt(MESSAGES, domain,purpose,aiCapability,aiUser,aiSubject,description,HRAssessment):
    S = "test {}"
    messages = deepcopy(MESSAGES)
    messages[1]['content'] = messages[1]['content'].format(domain,purpose,aiCapability,aiUser,aiSubject,description,HRAssessment)
    return messages


## 3 APPLY / RUN

In [ ]:
cost = 0

start_time = time.time()
i = 0

useElements = HR_risks
useI = str(useElements['id'])

print (f" Parsing use {useI}")

# Variables for message placeholders
domain = useElements['Details'][0]
purpose = useElements['Details'][1]
aiCapability = useElements['Details'][2]
aiUser = useElements['Details'][3]
aiSubject = useElements['Details'][4]

description = useElements['Description']
HRAssessment = useElements['Assessment of impact on human rights']

# print (HRAssessment)

# Extracting "Use i" details
use_i_details = [domain, purpose, aiCapability, aiUser, aiSubject, description, HRAssessment]

print(use_i_details)

# adapt the prompt for useI
messages = format_prompt(MESSAGES, domain, purpose, aiCapability, aiUser, aiSubject, description, HRAssessment)

# run the prompt
# response = get_completion_from_messages(messages, temperature=0)
# print(response)

response, token_count = get_completion_and_token_count(messages, temperature=0)
res = token_count
cost_chunk = (res['prompt_tokens'] * 0.03  + res['completion_tokens'] * 0.06)/1000.0
cost += cost_chunk

response = ast.literal_eval(response)


combined_response = {}
combined_response["id"]= useI
combined_response["Details"] = use_i_details
for k, v in response.items():
  combined_response[k] = v
print (combined_response)


###############################
# save result
with open(f"HR_risks_mitigation.json", "w") as json_file:
    json.dump(combined_response, json_file, indent=4)  # 4 spaces of indentation
# Download the file to your local machine
files.download(f"HR_risks_mitigation.json")
end_time = time.time()

print(f"Execution time: {end_time - start_time:.5f} seconds")
print (f"TOTAL COST {cost}")

 Parsing use 64
['Law enforcement', 'Crime prevention', 'Identify potential crime hotspots', 'CCTV', 'Police departments, security agencies', 'The AI system is designed to be used in the domain of law enforcement with the purpose of crime prevention. It is capable of identifying potential crime hotspots and is used by CCTV, with police departments and security agencies being the subjects of the AI.', [{'Article 2': 'Mixed', 'Reasoning for Support for Article 2': 'The AI system can support Article 2 by providing equal protection to all individuals, regardless of their race, colour, sex, language, religion, political or other opinion, national or social origin, property, birth or other status.', 'Relevant text Support for from Article 2': 'Everyone is entitled to all the rights and freedoms set forth in this Declaration, without distinction of any kind...', 'Reasoning for Undermining for Article 2': 'However, if the AI system is not properly calibrated or if it is biased in its data coll

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Execution time: 45.04777 seconds
TOTAL COST 0.14484


# THE END